# Parameter estimation

This notebook performs the parameter estimation for the cMyBP-C models to experimental kinetic data (Figures 2C,E and S2-S7,S9,S10,S13-S15) from the publication:

Thomas Kampourakis, Saraswathi Ponnam, Daniel Koch (2023):
The cardiac myosin binding protein-C phosphorylation state as a function of multiple protein kinase and phosphatase activities 
Preprint available under: https://doi.org/10.1101/2023.02.24.529959 

Figures based on the results from this notebooks are generated by the script '**model_comparisons.py**'

To perform parameter estimation and refinement for a model, execute the cell **Step 0** first, then choose a model and execute cells **Step 1, Step 2** and **Step 3**. Results will be saved as numpy arrays.

Make sure the respective model files and the folder with the experimental data (in the subfolder "**\DataFormatted**" are in the same folder as this script.
To avoid errors, it is advisable to perform parameter estimation for different models separately, i.e. it is best to close and restart the notebook for each model.

**Step 0 (all models)**: load packages, define boundaries and initial guesses for parameter estimates, define settings for parameter estimation algorithms.

In [1]:
from basico import *
%pylab
%matplotlib inline
import sys
import numpy as np
if not '../..' in sys.path:
    sys.path.append('../..')

cf = 1.7 #conversion factor from SEM to SD for n=3

# RSK kcat = 1.8, Km 1.3e-6,

# boundaries for parameters shared by all models
low_up_bnd = [ 
              #kcat values
              [0.05, 50],                            #k1
              [0.1, 5],    [0.1, 5],                 #k2     k3
              [0.05, 50],  [0.05, 20],               #k4     k5
              [0.05, 20],  [0.05, 50],               #k6     k7
              [0.05, 20],  [0.05, 20],               #k8     k9
              [5, 5],  [1.6-cf*0.1, 1.6+cf*0.1],     #k10    k11
              [1.8-cf*0.1,1.8+cf*0.1],  [5, 5],      #k12    k13
              [0.01, 20],  [0.05, 20],               #k14    k15
              [5, 5],      [0.01, 20],               #k16    k17
              [0.05, 20],  [5, 5],                   #k18    k19
              [0.01, 20],  [0.05, 20],               #k20    k21
              [0.5, 2], [0.05, 20] ,                 #sF22   k23
              [0.05, 20],  [0.5, 2],                 #k24    sF25
              [0.05, 20],  [0.05, 20],               #k26    k27
              [0.05, 50],  [0.05, 20],               #k28    k29
              [0.05, 20],                            #k30
              
              #Km values
              [1e-6, 75e-6],                         #K1     
              [1e-4, 1e-3],  [1e-4, 1e-3],           #K2     K3
              [1e-6, 75e-6], [1e-6, 1e-3],           #K4     K5
              [1e-6, 1e-3],  [1e-6, 75e-6],          #K6     K7
              [1e-6, 1e-3],  [1e-6, 1e-3],           #K8     K9
    
              [3e-6, 3e-6],  [(44.1-cf*5.2)*1e-6,(44.1+cf*5.2)*1e-6],             #K10    K11
              [(43.4-cf*5.3)*1e-6,(43.4+cf*5.3)*1e-6],  [4.5e-6, 4.5e-6],         #K12    K13
    
              [1e-6, 1e-3],  [1e-6, 1e-3],           #K14    K15
              [6e-6, 6e-6],  [1e-6, 1e-3],           #K16    K17
              [1e-6, 1e-3],  [7.5e-6, 7.5e-6],       #K18    K19
              [1e-6, 1e-3],  [1e-6, 1e-3],           #K20    K21
              [0.5, 2],      [1e-6, 1e-3],           #sF22   K23
              [1e-6, 1e-3],  [0.5, 2],               #K24    sF25
              [1e-6, 1e-3],  [1e-6, 1e-3],           #K26    K27
              [1e-6, 75e-6], [1e-6, 1e-3],           #K28    K29
              [1e-6, 1e-3],                          #K30                                            
             ]

# initial values for parameters shared by all models
startVal =   [
              #kcat values
              6,             #k1
              0.7,  1.3,     #k2     k3
              6,  1,         #k4     k5
              1,  6,         #k6     k7
              1,  1,         #k8     k9
              5,  1.6,       #k10    k11
              1.8, 5,        #k12    k13
              1,  1,         #k14    k15
              5,  1,         #k16    k17
              1,  5,         #k18    k19
              1,  1,         #k20    k21
              1,  1,         #k22    k23
              1,  1,         #k24    k25
              1,  1,         #k26    k27
              1,  1,         #k28    k29
              1,             #k30
    
              #Km values
              5e-6,                  #K1
              196.5e-6,  218.9e-6,   #K2     K3
              5e-6,  40e-6,          #K4     K5
              40e-6,  5e-6,          #K6     K7
              40e-6,  40e-6,         #K8     K9
              3e-6,  44.1e-6,        #K10    K11
              43.4e-6, 4.5e-6,       #K12    K13
              40e-6,  40e-6,         #K14    K15
              6e-6,  40e-6,          #K16    K17
              40e-6,  7.5e-6,        #K18    K19
              40e-6,  40e-6,         #K20    K21
              1,  40e-6,             #sF22    K23
              40e-6,  1,             #K24    sF25
              40e-6,  40e-6,         #K26    K27
              5e-6,  40e-6,          #K28    K29
              40e-6,                 #K30                                            
             ]

#Settings for parameter estimation algorithms
settingsGA = {'method': {'name': PE.GENETIC_ALGORITHM, 'Number of Generations':250,'Population Size':25}}
settingsHJ = {'method': {'name': PE.HOOKE_JEEVES, 'Iteration Limit':8,'Tolerance':2e-03, 'Rho':0.3}}
settingsHJ2 = {'method': {'name': PE.HOOKE_JEEVES, 'Iteration Limit':16,'Tolerance':2e-03, 'Rho':0.3}}

F:\anaconda3\envs\basico\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using matplotlib backend: <object object at 0x000002585E1470D0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## Model 1: Reactions 1-32, Michaelis-Menten type rate laws only, all experiments

### Step 1: load model, define which parameters to estimate and range for estimates etc

In [2]:
from basico import *
%pylab
%matplotlib inline
import sys
import numpy as np
if not '../..' in sys.path:
    sys.path.append('../..')

load_model('cMyBPC_model1.cps')

#Defining which parameters to estimate and setting boundaries and start values
vNames_cps = []
for i in range(1,31):
    vNames_cps.append('Values[k'+str(i)+'].InitialValue')
for i in range(1,31):
    vNames_cps.append('Values[K'+str(i)+'].InitialValue')
    
vNames_cps = [x.replace('k22', 'scalingFactor_k22') for x in vNames_cps]
vNames_cps = [x.replace('K22', 'scalingFactor_K22') for x in vNames_cps]
vNames_cps = [x.replace('k25', 'scalingFactor_k25') for x in vNames_cps]
vNames_cps = [x.replace('K25', 'scalingFactor_K25') for x in vNames_cps]
    
fit_items = []
for i in range(len(vNames_cps)):
    fit_items.append({'name': vNames_cps[i], 'lower': low_up_bnd[i][0], 'upper': low_up_bnd[i][1], 'start': startVal[i]})


set_fit_parameters(fit_items)

#Show settings for fitting parameters
get_fit_parameters()

Using matplotlib backend: QtAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


,lower,upper,start,affected,cn
name,,,,,
Values[k1],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k1],Refe..."
Values[k2],0.1,5.0,0.700000,[],"CN=Root,Model=New Model,Vector=Values[k2],Refe..."
Values[k3],0.1,5.0,1.300000,[],"CN=Root,Model=New Model,Vector=Values[k3],Refe..."
Values[k4],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k4],Refe..."
Values[k5],0.05,20.0,1.000000,[],"CN=Root,Model=New Model,Vector=Values[k5],Refe..."
Values[k6],0.05,20.0,1.000000,[],"CN=Root,Model=New Model,Vector=Values[k6],Refe..."
Values[k7],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k7],Refe..."
Values[k8],0.05,20.0,1.000000,[],"CN=Root,Model=New Model,Vector=Values[k8],Refe..."
Values[k9],0.05,20.0,1.000000,[],"CN=Root,Model=New Model,Vector=Values[k9],Refe..."


### Step 2: repeated global search using genetic algorithm

In [3]:
from tqdm import tqdm

result = np.array([])
resultOld = np.array([])

try:
    resultOld = np.load('resultsPE_model1_GA.npy')
    print("Prior runs found. New results will be appended.")
except:
    print("No prior runs found.")

nruns = 5 # number of parameter runs to be performed

for i in tqdm(range(0,nruns)):
    currentParams = run_parameter_estimation(settings=settingsGA, update_model=False)
    estimates = np.array(currentParams['sol']) # estimates for parameters
    stats = get_fit_statistic()
    objFval = np.array(stats['obj']) # value of the objective function
    # Fill the result matrix:
    # if first run, generate horizontal array with run nr., objFunc value and resulting parameters to fill first row
    # if i-th run, perform vertical concatenation to fill i-th row
    if len(result) > 0: 
        result = np.vstack((result, np.hstack((np.array([i]), objFval, estimates))))
    else:
        result = np.hstack((np.array([i]), objFval, estimates))

if len(resultOld)>0:
    result = np.vstack((result,resultOld))
    result[:,0] = np.asarray(range(len(result)))
    
np.save('resultsPE_model1_GA.npy',result)  
print(result) #show result matrix

No prior runs found.


100%|██████████| 5/5 [34:19<00:00, 411.97s/it]


[[0.00000000e+00 6.48184096e-06 4.51652372e+01 9.32603072e-01
  1.19092741e+00 1.41904807e+01 6.91963778e-01 8.97827577e-01
  7.00037069e+00 2.17478894e+00 5.26621667e+00 5.00000000e+00
  1.56129011e+00 1.81629273e+00 5.00000000e+00 1.80880027e+00
  1.15435477e+01 5.00000000e+00 1.19458178e+00 6.84252954e-01
  5.00000000e+00 2.96289544e+00 3.35895846e+00 1.13835832e+00
  5.69617137e-01 5.11706042e+00 6.76661831e-01 2.24790028e+00
  5.04675280e+00 2.26541280e+00 6.82636564e-01 3.95275575e-01
  1.80702874e-05 6.33553196e-04 6.96612120e-04 3.23493312e-06
  7.85210051e-06 2.73871697e-06 5.27110312e-06 1.56826904e-05
  7.20731118e-05 3.00000000e-06 3.52600000e-05 4.33398371e-05
  4.50000000e-06 1.41593147e-05 1.93566028e-04 6.00000000e-06
  1.88743023e-05 8.48856091e-06 7.50000000e-06 3.13651549e-05
  8.94797795e-05 1.52307087e+00 1.93154869e-05 5.06489989e-04
  8.63235464e-01 2.96898224e-05 7.59195012e-05 3.03466258e-06
  1.83375708e-05 1.42848899e-05]
 [1.00000000e+00 6.33634600e-06 1.344

### Step 3: refine parametersets with local optimization using Hooke & Jeeves algorithm

In [3]:
from tqdm import tqdm

try:
    result = np.load('resultsPE_model1_GA.npy')
    print("Refining parameter estimates...")
except:
    print("No data for optimization found!")

nruns = len(result)

resultRefined = np.array([])

for i in tqdm(range(0,nruns)):
    #print('Refinement run #',i+1,',')
    
    #start parameter estimation with resulting parameter values from the global search
    
    fit_items = []
    for ii in range(len(vNames_cps)):
        fit_items.append({'name': vNames_cps[ii], 'lower': low_up_bnd[ii][0], 'upper': low_up_bnd[ii][1], 'start': result[i, 2+ii]})

    set_fit_parameters(fit_items)
    #test output to confirm parameter estimation run starts with new parameter values each time
    print('Starting PE run with parameter values: \n \n', get_fit_parameters()['start'], '\n')
    currentParams = run_parameter_estimation(settings=settingsHJ, update_model=False)
    estimates = np.array(currentParams['sol'])
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(resultRefined) > 0:
        resultRefined = np.vstack((resultRefined, np.hstack((np.array([i]), objFval, estimates))))
    else:
        resultRefined = np.hstack((np.array([i]), objFval, estimates))

print(resultRefined)
np.save('resultsPE_model1_HJ.npy',resultRefined)  

Refining parameter estimates...


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Starting PE run with parameter values: 
 
 name
Values[k1]     12.751984
Values[k2]      1.800000
Values[k3]      0.945997
Values[k4]      0.987626
Values[k5]     10.986436
                 ...    
Values[K27]     0.000018
Values[K28]     0.000025
Values[K29]     0.000002
Values[K30]     0.000045
Values[K31]     0.000012
Name: start, Length: 62, dtype: float64 



  2%|█▌                                                                             | 1/50 [12:41<10:21:51, 761.45s/it]

Starting PE run with parameter values: 
 
 name
Values[k1]     31.518730
Values[k2]      1.800000
Values[k3]      1.008510
Values[k4]      0.993719
Values[k5]     50.000000
                 ...    
Values[K27]     0.000043
Values[K28]     0.000059
Values[K29]     0.000009
Values[K30]     0.000353
Values[K31]     0.000897
Name: start, Length: 62, dtype: float64 



...

Starting PE run with parameter values: 
 
 name
Values[k1]     20.512724
Values[k2]      1.800000
Values[k3]      0.908789
Values[k4]      0.699139
Values[k5]     29.533682
                 ...    
Values[K27]     0.000032
Values[K28]     0.000005
Values[K29]     0.000006
Values[K30]     0.000025
Values[K31]     0.000009
Name: start, Length: 62, dtype: float64 



 98%|█████████████████████████████████████████████████████████████████████████████▍ | 49/50 [4:41:16<04:39, 279.01s/it]

Starting PE run with parameter values: 
 
 name
Values[k1]     10.716497
Values[k2]      1.800000
Values[k3]      1.493933
Values[k4]      0.186373
Values[k5]     17.258104
                 ...    
Values[K27]     0.000019
Values[K28]     0.000041
Values[K29]     0.000007
Values[K30]     0.000021
Values[K31]     0.000021
Name: start, Length: 62, dtype: float64 



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [4:43:37<00:00, 340.35s/it]

[[0.00000000e+00 5.29162914e-06 1.93281824e+01 ... 2.26653250e-06
  3.48333598e-04 7.15433131e-05]
 [1.00000000e+00 5.02920340e-06 2.55676262e+01 ... 6.72409852e-06
  2.39059975e-04 1.92040950e-05]
 [2.00000000e+00 5.28162961e-06 4.40405525e+01 ... 3.63718358e-06
  1.22930143e-05 1.95154061e-05]
 ...
 [4.70000000e+01 5.15109694e-06 4.92227643e+01 ... 5.22202120e-06
  2.21619595e-04 1.24739409e-05]
 [4.80000000e+01 5.02852614e-06 2.21896397e+01 ... 5.42490421e-06
  8.87383531e-05 4.30578422e-05]
 [4.90000000e+01 5.03165615e-06 2.28892596e+01 ... 1.00000000e-06
  5.19378141e-05 4.37592405e-05]]


## Model 1: Michaelis-Menten type rate laws only, PP1 time course experiments only

### Step 1: load model, define which parameters to estimate and range for estimates etc

In [39]:
from basico import *
%pylab
%matplotlib inline
import sys
import numpy as np
if not '../..' in sys.path:
    sys.path.append('../..')

load_model('cMyBPC_model1_PP1_only.cps')

#Defining which parameters to estimate and setting boundaries and start values
vNames_cps = []
for i in range(1,31):
    vNames_cps.append('Values[k'+str(i)+'].InitialValue')
for i in range(1,31):
    vNames_cps.append('Values[K'+str(i)+'].InitialValue')
    
vNames_cps = [x.replace('k22', 'scalingFactor_k22') for x in vNames_cps]
vNames_cps = [x.replace('K22', 'scalingFactor_K22') for x in vNames_cps]
vNames_cps = [x.replace('k25', 'scalingFactor_k25') for x in vNames_cps]
vNames_cps = [x.replace('K25', 'scalingFactor_K25') for x in vNames_cps]
    
fit_items = []
for i in range(len(vNames_cps)):
    fit_items.append({'name': vNames_cps[i], 'lower': low_up_bnd[i][0], 'upper': low_up_bnd[i][1], 'start': startVal[i]})


set_fit_parameters(fit_items)

#Show settings for fitting parameters
get_fit_parameters()

Using matplotlib backend: QtAgg
Populating the interactive namespace from numpy and matplotlib


,lower,upper,start,affected,cn
name,,,,,
Values[k1],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k1],Refe..."
Values[k2],1.8,1.8,1.800000,[],"CN=Root,Model=New Model,Vector=Values[k2],Refe..."
Values[k3],0.1,5.0,0.700000,[],"CN=Root,Model=New Model,Vector=Values[k3],Refe..."
Values[k4],0.1,5.0,1.300000,[],"CN=Root,Model=New Model,Vector=Values[k4],Refe..."
Values[k5],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k5],Refe..."
...,...,...,...,...,...
Values[K27],1e-06,0.001,0.000040,[],"CN=Root,Model=New Model,Vector=Values[K27],Ref..."
Values[K28],1e-06,0.001,0.000040,[],"CN=Root,Model=New Model,Vector=Values[K28],Ref..."
Values[K29],1e-06,7.5e-05,0.000005,[],"CN=Root,Model=New Model,Vector=Values[K29],Ref..."


### Step 2: repeated global search using genetic algorithm

In [4]:
from tqdm import tqdm

result = np.array([])
resultOld = np.array([])

try:
    resultOld = np.load('resultsPE_model1_PP1_only_GA.npy')
    print("Prior runs found. New results will be appended.")
except:
    print("No prior runs found.")

nruns = 10

for i in tqdm(range(0,nruns)):
    currentParams = run_parameter_estimation(settings=settingsGA, update_model=False)
    estimates = np.array(currentParams['sol']) 
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(result) > 0: 
        result = np.vstack((result, np.hstack((np.array([i]), objFval, estimates))))
    else:
        result = np.hstack((np.array([i]), objFval, estimates))

if len(resultOld)>0:
    result = np.vstack((result,resultOld))
    result[:,0] = np.asarray(range(len(result)))
    
np.save('resultsPE_model1_PP1_only_GA.npy',result)  
print(result) #show result matrix

Prior runs found. New results will be appended.


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [26:12<00:00, 39.30s/it]

[[0.00000000e+00 1.04742294e-06 2.73495260e+00 ... 5.33645493e-06
  2.52729878e-05 5.95773434e-05]
 [1.00000000e+00 1.08360860e-06 1.73911142e+00 ... 6.21084837e-06
  4.89652058e-05 1.01553786e-04]
 [2.00000000e+00 1.09233338e-06 6.25754263e-01 ... 3.42196096e-05
  3.51396031e-05 5.91701510e-05]
 ...
 [4.70000000e+01 1.05115249e-06 4.67309415e+00 ... 5.61189927e-06
  1.77700705e-05 3.89475774e-06]
 [4.80000000e+01 1.27010037e-06 2.53672624e-01 ... 5.74223799e-06
  8.84910888e-06 2.83750713e-06]
 [4.90000000e+01 1.23872056e-06 2.51561027e-01 ... 6.98205246e-06
  6.28630867e-05 9.48852674e-06]]


### Step 3: refine parametersets with local optimization using Hooke & Jeeves algorithm

In [19]:
from tqdm import tqdm

try:
    result = np.load('resultsPE_model1_PP1_only_GA.npy')
    print("Refining parameter estimates...")
except:
    print("No data for optimization found!")

nruns = len(result)

resultRefined = np.array([])

for i in tqdm(range(0,nruns)):
    #print('Refinement run #',i+1,',')
    fit_items = []
    for ii in range(len(vNames_cps)):
        fit_items.append({'name': vNames_cps[ii], 'lower': low_up_bnd[ii][0], 'upper': low_up_bnd[ii][1], 'start': result[i, 2+ii]})

    set_fit_parameters(fit_items)
    ##test output to confirm parameter estimation run starts with new parameter values each time
    #print('Starting PE run with parameter values: \n \n', get_fit_parameters()['start'], '\n')
    currentParams = run_parameter_estimation(settings=settingsHJ2, update_model=False)
    estimates = np.array(currentParams['sol'])
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(resultRefined) > 0:
        resultRefined = np.vstack((resultRefined, np.hstack((np.array([i]), objFval, estimates))))
    else:
        resultRefined = np.hstack((np.array([i]), objFval, estimates))

print(resultRefined)
np.save('resultsPE_model1_PP1_only_HJ.npy',resultRefined)  

Refining parameter estimates...


100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [1:07:34<00:00, 81.08s/it]

[[0.00000000e+00 9.12474088e-07 5.00000000e-02 ... 5.33645493e-06
  2.54191668e-04 5.95773434e-05]
 [1.00000000e+00 9.13607862e-07 5.00000000e-02 ... 6.21084837e-06
  3.40387014e-04 1.01553786e-04]
 [2.00000000e+00 9.16456358e-07 5.00000000e-02 ... 3.42196096e-05
  3.37123379e-04 5.91701510e-05]
 ...
 [4.70000000e+01 9.34576278e-07 5.00000000e-02 ... 5.61189927e-06
  1.11698932e-04 3.89475774e-06]
 [4.80000000e+01 1.19438958e-06 5.00000000e-02 ... 5.74223799e-06
  1.00000000e-06 2.83750713e-06]
 [4.90000000e+01 1.17744875e-06 5.00000000e-02 ... 6.98205246e-06
  9.97097611e-04 9.48852674e-06]]


## Model 2: Phenotypic model assuming activation of reaction 3, PP1 time course experiments only

### Step 1: load model, define which parameters to estimate and range for estimates etc

In [5]:
from basico import *
%pylab
%matplotlib inline
import sys
import numpy as np
if not '../..' in sys.path:
    sys.path.append('../..')

load_model('cMyBPC_model2_PP1_only.cps')

#Defining which parameters to estimate and setting boundaries and start values
vNames_cps = []
for i in range(1,31):
    vNames_cps.append('Values[k'+str(i)+'].InitialValue')
for i in range(1,31):
    vNames_cps.append('Values[K'+str(i)+'].InitialValue')
    
vNames_cps = [x.replace('k22', 'scalingFactor_k22') for x in vNames_cps]
vNames_cps = [x.replace('K22', 'scalingFactor_K22') for x in vNames_cps]
vNames_cps = [x.replace('k25', 'scalingFactor_k25') for x in vNames_cps]
vNames_cps = [x.replace('K25', 'scalingFactor_K25') for x in vNames_cps]

vNames_cps.append('Values[r2_actF].InitialValue')
vNames_cps.append('Values[r2_Ka].InitialValue')


low_up_bnd2 = low_up_bnd
low_up_bnd2.append([0, 100]) #r2_actF
low_up_bnd2.append([1e-6, 1]) #r2_Ka

startVal2 = startVal
startVal2.append(1) #r2_actF
startVal2.append(0.1) #r2_Ka


fit_items = []
for i in range(len(vNames_cps)):
    fit_items.append({'name': vNames_cps[i], 'lower': low_up_bnd2[i][0], 'upper': low_up_bnd2[i][1], 'start': startVal2[i]})


set_fit_parameters(fit_items)

#Show settings for fitting parameters
get_fit_parameters()

Using matplotlib backend: QtAgg
Populating the interactive namespace from numpy and matplotlib


,lower,upper,start,affected,cn
name,,,,,
Values[k1],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k1],Refe..."
Values[k2],1.8,1.8,1.800000,[],"CN=Root,Model=New Model,Vector=Values[k2],Refe..."
Values[k3],0.1,5.0,0.700000,[],"CN=Root,Model=New Model,Vector=Values[k3],Refe..."
Values[k4],0.1,5.0,1.300000,[],"CN=Root,Model=New Model,Vector=Values[k4],Refe..."
Values[k5],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k5],Refe..."
...,...,...,...,...,...
Values[K29],1e-06,7.5e-05,0.000005,[],"CN=Root,Model=New Model,Vector=Values[K29],Ref..."
Values[K30],1e-06,0.001,0.000040,[],"CN=Root,Model=New Model,Vector=Values[K30],Ref..."
Values[K31],1e-06,0.001,0.000040,[],"CN=Root,Model=New Model,Vector=Values[K31],Ref..."


### Step 2: repeated global search using genetic algorithm

In [6]:
from tqdm import tqdm

result = np.array([])
resultOld = np.array([])

try:
    resultOld = np.load('resultsPE_model2_PP1_only_GA.npy')
    print("Prior runs found. New results will be appended.")
except:
    print("No prior runs found.")

nruns = 20

for i in tqdm(range(0,nruns)):
    currentParams = run_parameter_estimation(settings=settingsGA, update_model=False)
    estimates = np.array(currentParams['sol']) 
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(result) > 0: 
        result = np.vstack((result, np.hstack((np.array([i]), objFval, estimates))))
    else:
        result = np.hstack((np.array([i]), objFval, estimates))

if len(resultOld)>0:
    result = np.vstack((result,resultOld))
    result[:,0] = np.asarray(range(len(result)))
    
np.save('resultsPE_model2_PP1_only_GA.npy',result)  
print(result) #show result matrix

Prior runs found. New results will be appended.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [17:59<00:00, 53.98s/it]

[[0.00000000e+000 1.07275510e-006 3.12085837e+001 ... 2.70675330e-004
  3.68520619e-058 5.52755424e-002]
 [1.00000000e+000 8.31760219e-007 6.99300512e+000 ... 2.45972768e-005
  4.41748987e+000 9.54993109e-002]
 [2.00000000e+000 1.11994010e-006 4.27538042e+000 ... 1.15789206e-006
  2.00543525e-045 1.00000000e-006]
 ...
 [4.70000000e+001 9.91450965e-007 4.00436609e+001 ... 9.31697176e-006
  4.92113042e-299 3.11777067e-005]
 [4.80000000e+001 9.47824113e-007 7.17446544e-002 ... 4.44434599e-006
  2.90677727e+000 7.11752289e-002]
 [4.90000000e+001 1.01215606e-006 1.97252403e-001 ... 8.06169481e-006
  3.64560808e-042 7.16450804e-003]]


### Step 3: refine parametersets with local optimization using Hooke & Jeeves algorithm

In [ ]:
from tqdm import tqdm

try:
    result = np.load('resultsPE_model2_PP1_only_GA.npy')
    print("Refining parameter estimates...")
except:
    print("No data for optimization found!")

nruns = len(result)

resultRefined = np.array([])

for i in tqdm(range(0,nruns)):
    #print('Refinement run #',i+1,',')
    fit_items = []
    for ii in range(len(vNames_cps)):
        fit_items.append({'name': vNames_cps[ii], 'lower': low_up_bnd2[ii][0], 'upper': low_up_bnd2[ii][1], 'start': result[i, 2+ii]})

    set_fit_parameters(fit_items)
    #test output to confirm parameter estimation run starts with new parameter values each time
    #print('Starting PE run with parameter values: \n \n', get_fit_parameters()['start'], '\n')
    currentParams = run_parameter_estimation(settings=settingsHJ2, update_model=False)
    estimates = np.array(currentParams['sol'])
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(resultRefined) > 0:
        resultRefined = np.vstack((resultRefined, np.hstack((np.array([i]), objFval, estimates))))
    else:
        resultRefined = np.hstack((np.array([i]), objFval, estimates))

print(resultRefined)
np.save('resultsPE_model2_PP1_only_HJ.npy',resultRefined)  

Refining parameter estimates...


 68%|███████████████████████████████████████████████████████▊                          | 34/50 [48:07<22:46, 85.42s/it]

## Model 3: Allosteric activation of reaction 3, PP1 time course experiments only

### Step 1: load model, define which parameters to estimate and range for estimates etc

In [2]:
from basico import *
%pylab
%matplotlib inline
import sys
import numpy as np
if not '../..' in sys.path:
    sys.path.append('../..')

load_model('cMyBPC_model3_PP1_only.cps')

#Defining which parameters to estimate and setting boundaries and start values
vNames_cps = []
for i in range(1,31):
    vNames_cps.append('Values[k'+str(i)+'].InitialValue')
for i in range(1,31):
    vNames_cps.append('Values[K'+str(i)+'].InitialValue')
    
vNames_cps = [x.replace('k22', 'scalingFactor_k22') for x in vNames_cps]
vNames_cps = [x.replace('K22', 'scalingFactor_K22') for x in vNames_cps]
vNames_cps = [x.replace('k25', 'scalingFactor_k25') for x in vNames_cps]
vNames_cps = [x.replace('K25', 'scalingFactor_K25') for x in vNames_cps]

vNames_cps.append('Values[alpha_r2].InitialValue')    
vNames_cps.append('Values[ka_2].InitialValue')    
vNames_cps.append('Values[Ka_2].InitialValue')    

low_up_bnd2 = low_up_bnd
low_up_bnd2.append([0.1, 50]) #alpha_r2
low_up_bnd2.append([0.01, 50]) #ka_2
low_up_bnd2.append([1e-6, 1e-3]) #Ka_2


startVal2 = startVal
startVal2.append(1) #alpha_r2
startVal2.append(1) #ka_2
startVal2.append(1e-5) #Ka_2


fit_items = []
for i in range(len(vNames_cps)):
    fit_items.append({'name': vNames_cps[i], 'lower': low_up_bnd2[i][0], 'upper': low_up_bnd2[i][1], 'start': startVal2[i]})

set_fit_parameters(fit_items)

#Show settings for fitting parameters
get_fit_parameters()

Using matplotlib backend: QtAgg
Populating the interactive namespace from numpy and matplotlib


,lower,upper,start,affected,cn
name,,,,,
Values[k1],0.05,50.0,6.00000,[],"CN=Root,Model=New Model,Vector=Values[k1],Refe..."
Values[k2],1.8,1.8,1.80000,[],"CN=Root,Model=New Model,Vector=Values[k2],Refe..."
Values[k3],0.1,5.0,0.70000,[],"CN=Root,Model=New Model,Vector=Values[k3],Refe..."
Values[k4],0.1,5.0,1.30000,[],"CN=Root,Model=New Model,Vector=Values[k4],Refe..."
Values[k5],0.05,50.0,6.00000,[],"CN=Root,Model=New Model,Vector=Values[k5],Refe..."
...,...,...,...,...,...
Values[K30],1e-06,0.001,0.00004,[],"CN=Root,Model=New Model,Vector=Values[K30],Ref..."
Values[K31],1e-06,0.001,0.00004,[],"CN=Root,Model=New Model,Vector=Values[K31],Ref..."
Values[alpha_r3],0.1,50.0,1.00000,[],"CN=Root,Model=New Model,Vector=Values[alpha_r3..."


### Step 2: repeated global search using genetic algorithm

In [9]:
from tqdm import tqdm

result = np.array([])
resultOld = np.array([])

try:
    resultOld = np.load('resultsPE_model3_PP1_only_GA.npy')
    print("Prior runs found. New results will be appended.")
except:
    print("No prior runs found.")

nruns = 20

for i in tqdm(range(0,nruns)):
    currentParams = run_parameter_estimation(settings=settingsGA, update_model=False)
    estimates = np.array(currentParams['sol']) 
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(result) > 0: 
        result = np.vstack((result, np.hstack((np.array([i]), objFval, estimates))))
    else:
        result = np.hstack((np.array([i]), objFval, estimates))

if len(resultOld)>0:
    result = np.vstack((result,resultOld))
    result[:,0] = np.asarray(range(len(result)))
    
np.save('resultsPE_model3_PP1_only_GA.npy',result)  
print(result) #show result matrix

Prior runs found. New results will be appended.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [16:37<00:00, 49.87s/it]

[[0.00000000e+00 8.15164345e-07 6.04653982e+00 ... 1.22407381e-01
  1.23095514e+00 2.61405083e-06]
 [1.00000000e+00 7.70727390e-07 8.98281328e+00 ... 5.93749889e+00
  1.85754420e+01 2.98855486e-06]
 [2.00000000e+00 9.81421274e-07 2.31348552e+01 ... 7.74706204e-01
  2.53011753e+00 1.29654322e-05]
 ...
 [4.70000000e+01 8.50277255e-07 1.52017530e+01 ... 2.82731201e+00
  1.27361498e+01 1.77433268e-06]
 [4.80000000e+01 9.51994701e-07 9.80891934e+00 ... 2.85996074e-01
  8.61785258e+00 4.81287458e-05]
 [4.90000000e+01 1.16877276e-06 9.06642626e-01 ... 1.15608996e-01
  3.30151959e+00 5.25497402e-05]]


### Step 3: refine parametersets with local optimization using Hooke & Jeeves algorithm

In [3]:
from tqdm import tqdm

try:
    result = np.load('resultsPE_model3_PP1_only_GA.npy')
    print("Refining parameter estimates...")
except:
    print("No data for optimization found!")

nruns = len(result)

resultRefined = np.array([])

for i in tqdm(range(0,nruns)):
    #print('Refinement run #',i+1,',')
    fit_items = []
    for ii in range(len(vNames_cps)):
        fit_items.append({'name': vNames_cps[ii], 'lower': low_up_bnd2[ii][0], 'upper': low_up_bnd2[ii][1], 'start': result[i, 2+ii]})

    set_fit_parameters(fit_items)
    #test output to confirm parameter estimation run starts with new parameter values each time
    #print('Starting PE run with parameter values: \n \n', get_fit_parameters()['start'], '\n')
    currentParams = run_parameter_estimation(settings=settingsHJ2, update_model=False)
    estimates = np.array(currentParams['sol'])
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(resultRefined) > 0:
        resultRefined = np.vstack((resultRefined, np.hstack((np.array([i]), objFval, estimates))))
    else:
        resultRefined = np.hstack((np.array([i]), objFval, estimates))

print(resultRefined)
np.save('resultsPE_model3_PP1_only_HJ.npy',resultRefined)  

Refining parameter estimates...


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [52:19<00:00, 62.78s/it]

[[0.00000000e+00 6.80470485e-07 5.00000000e-02 ... 2.88484110e-01
  6.72881933e+00 6.98098482e-05]
 [1.00000000e+00 6.29610011e-07 5.00000000e-02 ... 2.14450585e+00
  2.96644235e+01 1.32779998e-05]
 [2.00000000e+00 8.95377868e-07 5.00000000e-02 ... 1.44928163e+00
  2.75145221e+00 1.00248721e-06]
 ...
 [4.70000000e+01 7.29379037e-07 5.00000000e-02 ... 2.23264347e+00
  1.09823819e+01 2.45397692e-06]
 [4.80000000e+01 9.01941308e-07 5.00000000e-02 ... 1.99356424e-01
  1.30442402e+01 1.81545961e-04]
 [4.90000000e+01 9.92359647e-07 5.00000000e-02 ... 1.08146966e-01
  7.69528090e+00 1.14836947e-04]]


## Model 4: Structural transition model, PP1 time course experiments only

### Step 1: load model, define which parameters to estimate and range for estimates etc

In [2]:
from basico import *
%pylab
%matplotlib inline
import sys
import numpy as np
if not '../..' in sys.path:
    sys.path.append('../..')

load_model('cMyBPC_model4_PP1_only.cps')

#Defining which parameters to estimate and setting boundaries and start values
vNames_cps = []
for i in range(1,31):
    vNames_cps.append('Values[k'+str(i)+'].InitialValue')
for i in range(1,31):
    vNames_cps.append('Values[K'+str(i)+'].InitialValue')
    
vNames_cps = [x.replace('k22', 'scalingFactor_k22') for x in vNames_cps]
vNames_cps = [x.replace('K22', 'scalingFactor_K22') for x in vNames_cps]
vNames_cps = [x.replace('k25', 'scalingFactor_k25') for x in vNames_cps]
vNames_cps = [x.replace('K25', 'scalingFactor_K25') for x in vNames_cps]

vNames_cps.append('Values[k2f].InitialValue')    
vNames_cps.append('Values[K2f].InitialValue')    
vNames_cps.append('Values[kisoF].InitialValue')    
vNames_cps.append('Values[kisoR].InitialValue')  

low_up_bnd2 = low_up_bnd
low_up_bnd2.append([0.1, 50]) #k2f
low_up_bnd2.append([1e-6, 300e-6]) #K2f
low_up_bnd2.append([1e-3, 1e5]) #kisoF
low_up_bnd2.append([0, 1]) #kisoR

startVal2 = startVal
startVal2.append(1) #k2f
startVal2.append(4e-5) #K2f
startVal2.append(1) #kisoF
startVal2.append(1e-6) #kisoR

fit_items = []
for i in range(len(vNames_cps)):
    fit_items.append({'name': vNames_cps[i], 'lower': low_up_bnd2[i][0], 'upper': low_up_bnd2[i][1], 'start': startVal2[i]})

set_fit_parameters(fit_items)

#Show settings for fitting parameters
get_fit_parameters()

Using matplotlib backend: QtAgg
Populating the interactive namespace from numpy and matplotlib


,lower,upper,start,affected,cn
name,,,,,
Values[k1],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k1],Refe..."
Values[k2],1.8,1.8,1.800000,[],"CN=Root,Model=New Model,Vector=Values[k2],Refe..."
Values[k3],0.1,5.0,0.700000,[],"CN=Root,Model=New Model,Vector=Values[k3],Refe..."
Values[k4],0.1,5.0,1.300000,[],"CN=Root,Model=New Model,Vector=Values[k4],Refe..."
Values[k5],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k5],Refe..."
...,...,...,...,...,...
Values[K31],1e-06,0.001,0.000040,[],"CN=Root,Model=New Model,Vector=Values[K31],Ref..."
Values[k3f],0.1,50.0,1.000000,[],"CN=Root,Model=New Model,Vector=Values[k3f],Ref..."
Values[K3f],1e-06,0.0003,0.000040,[],"CN=Root,Model=New Model,Vector=Values[K3f],Ref..."


### Step 2: repeated global search using genetic algorithm

In [5]:
from tqdm import tqdm

result = np.array([])
resultOld = np.array([])

try:
    resultOld = np.load('resultsPE_model4_PP1_only_GA.npy')
    print("Prior runs found. New results will be appended.")
except:
    print("No prior runs found.")

nruns = 1

for i in tqdm(range(0,nruns)):
    currentParams = run_parameter_estimation(settings=settingsGA, update_model=False)
    estimates = np.array(currentParams['sol']) 
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(result) > 0: 
        result = np.vstack((result, np.hstack((np.array([i]), objFval, estimates))))
    else:
        result = np.hstack((np.array([i]), objFval, estimates))

if len(resultOld)>0:
    result = np.vstack((result,resultOld))
    result[:,0] = np.asarray(range(len(result)))
    
np.save('resultsPE_model4_PP1_only_GA.npy',result)  
print(result) #show result matrix

Prior runs found. New results will be appended.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:10<00:00, 70.21s/it]

[[0.00000000e+000 7.94292224e-007 2.69260008e+001 1.80000000e+000
  5.65433282e-001 1.02715477e+000 7.56344232e-001 1.44394251e+001
  6.06476334e+000 1.48567489e-001 7.40903874e+000 2.00000000e+001
  5.00000000e+000 1.59177240e+000 1.63000000e+000 5.00000000e+000
  1.70609492e+001 4.58269511e+000 5.00000000e+000 2.30987129e+000
  5.99569793e+000 5.00000000e+000 1.62232361e+000 2.61289884e+000
  5.00000000e-001 1.43567790e+000 7.88349800e-001 7.19994657e-001
  2.39340880e+000 9.79383392e-001 5.12621237e+000 3.72445479e+000
  1.07918679e+000 1.28363903e-006 1.30000000e-006 7.26159326e-004
  7.23334943e-004 5.91691526e-006 2.99667906e-004 2.70161581e-006
  3.12089464e-006 2.03377470e-004 1.44139665e-005 3.00000000e-006
  5.10012909e-005 4.91495163e-005 4.50000000e-006 3.94575471e-004
  1.59079795e-004 6.00000000e-006 3.18566687e-005 2.96955188e-006
  7.50000000e-006 3.24132396e-004 4.50342569e-005 1.36058597e+000
  6.29553774e-005 7.97678915e-006 1.25527840e+000 2.28541152e-004
  1.365770

### Step 3: refine parametersets with local optimization using Hooke & Jeeves algorithm

In [6]:
from tqdm import tqdm

try:
    result = np.load('resultsPE_model4_PP1_only_GA.npy')
    print("Refining parameter estimates...")
except:
    print("No data for optimization found!")

nruns = len(result)

resultRefined = np.array([])

for i in tqdm(range(0,nruns)):
    #print('Refinement run #',i+1,',')
    fit_items = []
    for ii in range(len(vNames_cps)):
        fit_items.append({'name': vNames_cps[ii], 'lower': low_up_bnd2[ii][0], 'upper': low_up_bnd2[ii][1], 'start': result[i, 2+ii]})

    set_fit_parameters(fit_items)
    #test output to confirm parameter estimation run starts with new parameter values each time
    #print('Starting PE run with parameter values: \n \n', get_fit_parameters()['start'], '\n')
    currentParams = run_parameter_estimation(settings=settingsHJ2, update_model=False)
    estimates = np.array(currentParams['sol'])
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(resultRefined) > 0:
        resultRefined = np.vstack((resultRefined, np.hstack((np.array([i]), objFval, estimates))))
    else:
        resultRefined = np.hstack((np.array([i]), objFval, estimates))

print(resultRefined)
np.save('resultsPE_model4_PP1_only_HJ.npy',resultRefined)  

Refining parameter estimates...


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [03:24<00:00, 102.13s/it]

[[0.00000000e+000 6.77244030e-007 5.00000000e-002 1.80000000e+000
  6.66600605e-001 1.02715477e+000 7.56344232e-001 1.67682156e+001
  6.06476334e+000 1.48567489e-001 7.97360749e-001 2.00000000e+001
  5.00000000e+000 1.77000000e+000 1.63000000e+000 5.00000000e+000
  1.68338290e+001 4.58269511e+000 5.00000000e+000 3.51666354e+000
  5.99569793e+000 5.00000000e+000 1.00000000e-002 2.61289884e+000
  5.00000000e-001 1.95097141e+000 7.88349800e-001 7.19994657e-001
  5.00000000e-002 9.79383392e-001 5.12621237e+000 1.01790839e+001
  1.07918679e+000 1.28363903e-006 1.30000000e-006 7.23218381e-004
  7.23334943e-004 5.91691526e-006 3.09134415e-004 2.70161581e-006
  3.12089464e-006 1.75555432e-005 1.44139665e-005 3.00000000e-006
  3.52600000e-005 4.91495163e-005 4.50000000e-006 3.87544136e-004
  1.59079795e-004 6.00000000e-006 4.93517141e-005 2.96955188e-006
  7.50000000e-006 1.00000000e-003 4.50342569e-005 1.36058597e+000
  1.18742656e-004 7.97678915e-006 1.25527840e+000 9.41390717e-004
  1.365770

## Model 4: Structural transition model, PP1 and PP2A time course experiments
### Step 1: load model, define which parameters to estimate and range for estimates etc

In [3]:
from basico import *
%pylab
%matplotlib inline
import sys
import numpy as np
if not '../..' in sys.path:
    sys.path.append('../..')

load_model('cMyBPC_model4_PP1,PP2A.cps')

#Defining which parameters to estimate and setting boundaries and start values
vNames_cps = []
for i in range(1,31):
    vNames_cps.append('Values[k'+str(i)+'].InitialValue')
for i in range(1,31):
    vNames_cps.append('Values[K'+str(i)+'].InitialValue')
    
vNames_cps = [x.replace('k22', 'scalingFactor_k22') for x in vNames_cps]
vNames_cps = [x.replace('K22', 'scalingFactor_K22') for x in vNames_cps]
vNames_cps = [x.replace('k25', 'scalingFactor_k25') for x in vNames_cps]
vNames_cps = [x.replace('K25', 'scalingFactor_K25') for x in vNames_cps]

vNames_cps.append('Values[k2f].InitialValue')    
vNames_cps.append('Values[K2f].InitialValue')  
vNames_cps.append('Values[kisoF].InitialValue')    
vNames_cps.append('Values[kisoR].InitialValue')
vNames_cps.append('Values[k3f].InitialValue')    
vNames_cps.append('Values[K3f].InitialValue') 

low_up_bnd2 = low_up_bnd
low_up_bnd2.append([0.1, 50]) #k2f
low_up_bnd2.append([1e-6, 300e-6]) #K2f
low_up_bnd2.append([1e-3, 1e5]) #kisoF
low_up_bnd2.append([0, 1]) #kisoR
low_up_bnd2.append([0.1, 50]) #k3f
low_up_bnd2.append([1e-6, 300e-6]) #K3f

startVal2 = startVal
startVal2.append(1) #k2f
startVal2.append(4e-5) #K2f
startVal2.append(1) #kisoF
startVal2.append(1e-6) #kisoR
startVal2.append(1) #k3f
startVal2.append(4e-5) #K3f

fit_items = []
for i in range(len(vNames_cps)):
    fit_items.append({'name': vNames_cps[i], 'lower': low_up_bnd2[i][0], 'upper': low_up_bnd2[i][1], 'start': startVal2[i]})
      
set_fit_parameters(fit_items)

#Show settings for fitting parameters
get_fit_parameters()


Using matplotlib backend: QtAgg
Populating the interactive namespace from numpy and matplotlib


,lower,upper,start,affected,cn
name,,,,,
Values[k1],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k1],Refe..."
Values[k2],1.8,1.8,1.800000,[],"CN=Root,Model=New Model,Vector=Values[k2],Refe..."
Values[k3],0.1,5.0,0.700000,[],"CN=Root,Model=New Model,Vector=Values[k3],Refe..."
Values[k4],0.1,5.0,1.300000,[],"CN=Root,Model=New Model,Vector=Values[k4],Refe..."
Values[k5],0.05,50.0,6.000000,[],"CN=Root,Model=New Model,Vector=Values[k5],Refe..."
...,...,...,...,...,...
Values[K3f],1e-06,0.0003,0.000040,[],"CN=Root,Model=New Model,Vector=Values[K3f],Ref..."
Values[kisoF],0.001,100000.0,1.000000,[],"CN=Root,Model=New Model,Vector=Values[kisoF],R..."
Values[kisoR],0.0,1.0,0.000001,[],"CN=Root,Model=New Model,Vector=Values[kisoR],R..."


### Step 2: repeated global search using genetic algorithm

In [4]:
from tqdm import tqdm

result = np.array([])
resultOld = np.array([])

idx_incl = [2,3,4,8,10,11,12,13,14,15,16,22,26,27,29,30,31,34,38,40,42,44,46,47,48]
rxsPP1 = [2,5,8,11,14,17,20,23,26,29]
rxsmod4 = [62,63,64,65]

startValsPP1 = np.load('resultsPE_model4_PP1_only_HJn.npy')  
startValsPP1 = startValsPP1[:,2:]

try:
    resultOld = np.load('resultsPE_model4_PP1,PP2A_GA2.npy')
    print("Prior runs found. New results will be appended.")
except:
    print("No prior runs found.")

nruns = 25

idx = 0
for i in tqdm(range(0,nruns)):
    for ii in range(len(rxsPP1)):
        fit_items[rxsPP1[ii]-1]['start'] = startValsPP1[idx_incl[idx],rxsPP1[ii]-1]
        fit_items[rxsPP1[ii]+30]['start'] = startValsPP1[idx_incl[idx],rxsPP1[ii]+30]
    for ii in range(len(rxsmod4)):
        fit_items[rxsmod4[ii]-1]['start'] = startValsPP1[idx_incl[idx],rxsmod4[ii]-1]
    set_fit_parameters(fit_items)
    currentParams = run_parameter_estimation(settings=settingsGA, update_model=False)
    estimates = np.array(currentParams['sol']) 
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    idx += 1
    if idx > len(idx_incl):
        idx = 0
    if len(result) > 0: 
        result = np.vstack((result, np.hstack((np.array([i]), objFval, estimates))))
    else:
        result = np.hstack((np.array([i]), objFval, estimates))

if len(resultOld)>0:
    result = np.vstack((result,resultOld))
    result[:,0] = np.asarray(range(len(result)))
    
np.save('resultsPE_model4_PP1,PP2A_GA2.npy',result)  
print(result) #show result matrix

Prior runs found. New results will be appended.


100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [2:26:54<00:00, 352.60s/it]

[[0.00000000e+000 6.95337748e-006 1.92959003e-001 ... 1.87463903e-188
  1.25528546e+001 1.84500809e-005]
 [1.00000000e+000 4.57676281e-006 3.18043667e+000 ... 3.01462705e-007
  4.51232811e+000 5.20101552e-006]
 [2.00000000e+000 4.23927329e-006 4.92673354e+000 ... 3.83955732e-147
  7.31044051e-001 1.28850963e-004]
 ...
 [4.70000000e+001 4.00274252e-006 4.03437878e+000 ... 3.82429671e-192
  1.00000000e-001 1.16331423e-006]
 [4.80000000e+001 4.84560247e-006 6.94586105e+000 ... 1.75611947e-169
  8.20217368e-001 1.19915568e-005]
 [4.90000000e+001 3.97246821e-006 1.34155575e+000 ... 7.87080412e-010
  1.25813135e+000 1.27419960e-005]]


### Step 3: refine parametersets with local optimization using Hooke & Jeeves algorithm

In [5]:
from tqdm import tqdm

try:
    result = np.load('resultsPE_model4_PP1,PP2A_GA2.npy')
    print("Refining parameter estimates...")
except:
    print("No data for optimization found!")

nruns = len(result)

resultRefined = np.array([])

for i in tqdm(range(0,nruns)):
    #print('Refinement run #',i+1,',')
    fit_items = []
    for ii in range(len(vNames_cps)):
        fit_items.append({'name': vNames_cps[ii], 'lower': low_up_bnd2[ii][0], 'upper': low_up_bnd2[ii][1], 'start': result[i, 2+ii]})
    set_fit_parameters(fit_items)
    #test output to confirm parameter estimation run starts with new parameter values each time
    #print('Starting PE run with parameter values: \n \n', get_fit_parameters()['start'], '\n')
    currentParams = run_parameter_estimation(settings=settingsHJ, update_model=False)
    estimates = np.array(currentParams['sol'])
    stats = get_fit_statistic()
    objFval = np.array(stats['obj'])
    if len(resultRefined) > 0:
        resultRefined = np.vstack((resultRefined, np.hstack((np.array([i]), objFval, estimates))))
    else:
        resultRefined = np.hstack((np.array([i]), objFval, estimates))

print(resultRefined)
np.save('resultsPE_model4_PP1,PP2A_HJ2.npy',resultRefined)  

Refining parameter estimates...


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [6:00:14<00:00, 432.29s/it]

[[0.00000000e+000 5.72584362e-006 5.00000000e-002 ... 1.87463903e-188
  3.88987858e+000 2.78850833e-005]
 [1.00000000e+000 3.55937987e-006 5.00000000e-002 ... 0.00000000e+000
  1.21431262e+000 4.04670213e-006]
 [2.00000000e+000 3.48182338e-006 5.00000000e-002 ... 3.83955732e-147
  2.02536485e+000 1.00000000e-006]
 ...
 [4.70000000e+001 3.34140147e-006 5.00000000e-002 ... 3.82429671e-192
  1.00000000e-001 3.23445561e-006]
 [4.80000000e+001 4.08355590e-006 5.00000000e-002 ... 1.75611947e-169
  4.07566010e-001 4.16037471e-005]
 [4.90000000e+001 3.31113768e-006 5.00000000e-002 ... 1.35453391e-009
  5.01101136e-001 2.15033925e-005]]
